In [1]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# goals
# - line history
# - % bets
# - % money
# NFL side and total

In [13]:
# scraping function
def scrape_for_soup(base_url, url):
    # send GET request
    response = requests.get(base_url+url)

    # status check
    if response.status_code != 200:
        return(f"Error: {response.status_code}")

    # parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [3]:
base_url = 'https://www.sportsbookreview.com'
url = '/betting-odds/nfl-football/'
soup = scrape_for_soup(base_url, url)

# parse links for game spread and spread history
games = soup.find_all('div', id='tbody-nfl')
main_links = set()
line_links = set()
for game in games:
    for a_tag in game.find_all('a', href=True):
        link = a_tag['href']
        if 'scores' in link:
            main_links.add(link)
        if 'line-history' in link:
            line_links.add(link)

In [4]:
main_links

{'/scores/nfl-football/matchup/304807/',
 '/scores/nfl-football/matchup/304842/',
 '/scores/nfl-football/matchup/304916/',
 '/scores/nfl-football/matchup/304917/',
 '/scores/nfl-football/matchup/304918/',
 '/scores/nfl-football/matchup/304920/',
 '/scores/nfl-football/matchup/304921/',
 '/scores/nfl-football/matchup/304922/',
 '/scores/nfl-football/matchup/304932/',
 '/scores/nfl-football/matchup/304935/',
 '/scores/nfl-football/matchup/304936/',
 '/scores/nfl-football/matchup/304937/',
 '/scores/nfl-football/matchup/304941/',
 '/scores/nfl-football/matchup/304945/',
 '/scores/nfl-football/matchup/304946/'}

In [27]:
# iterate through each link to scrape target data
game_url = list(main_links)[0]
# send request
game_soup = scrape_for_soup(base_url, game_url)
# pull number bets on spread and o/u

In [6]:
print(base_url+game_url)

https://www.sportsbookreview.com/scores/nfl-football/matchup/304917/


In [35]:
# spread bets
spread = game_soup.find('div', text=re.compile('Spread'))
home_perc = spread.next_sibling.get_text()[:-1]
away_perc = spread.previous_sibling.get_text()[:-1]

# over/under bets
total_bets = game_soup.find('div', text=re.compile('Total'))
over_perc = total_bets.previous_sibling.get_text()[:-3]
under_perc = total_bets.next_sibling.get_text()[:-3]
# pull total
tot_div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
# find the span that contains the best total for the over and strip the first element (second element is odds if needed)
total = float(tot_div.find('h4', text='Over').find_parent('div').find_next_sibling('div').find('span').contents[0].strip())

# teams and spread
away_div = game_soup.find('div', class_=re.compile('TeamName'))
home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
print(f'The spread is {home_spread}')
print(f'% bets {home_team} {home_spread}: {home_perc}')
print(f'% bets {away_team} {away_spread} % bets: {away_perc} \n')
print(f'The total is {str(total)}')
print(f'% bets over {str(total)}: {over_perc}')
print(f'% bets under {str(total)}:{under_perc}')

The spread is +2
% bets New OrleansSaints +2: 63
% bets DenverBroncos -2 % bets: 37 

The total is 36.5
% bets over 36.5: 65
% bets under 36.5:35


In [ ]:
#TODO
# iterate through links and add these to df

In [9]:
# next: pull total history/movement if possible
line_url = list(line_links)[0]
line_soup = scrape_for_soup(base_url, line_url)

In [10]:
print(base_url+line_url)

https://www.sportsbookreview.com/betting-odds/nfl-football/line-history/304842/


In [26]:
div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
print(div)

None
